### chemcalc API와 실제데이터를 활용해 가능한 분자 구조를 찾아보고 자동으로 그려보쟈!

#### 1. pandas를 활용해 데이터를 불러오고 전처리

In [3]:
import pandas as pd

In [4]:
df = pd.read_excel('data/Example 1.xlsx',index_col = False)

In [5]:
df

,1,85.0281,15505,92.6,3588,1.7,0.0055,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,2,85.1014,16848,16.6,643,0.3,0.0051,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,86.0599,15149,19.4,753,0.4,0.0057,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,87.0438,18891,17,663,0.3,0.0046,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,88.0755,15455,55.3,2156,1.1,0.0057,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6,89.0593,17062,5.5,215,0.1,0.0052,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2656,2654,1001.8604,15828,1.8,227,0.1,0.0633,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2657,2655,1002.8672,16946,2.5,310,0.2,0.0592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2658,2656,1003.8993,16053,2.3,290,0.1,0.0625,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2659,691,354.1386+'[Sorting_0529_Fe Complexes .xlsx]#4 ...,12797,27.1,2249,1.1,0.0277,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df = df.iloc[:,1:7]

In [7]:
df = df.dropna()

In [8]:
df.columns = ['var1','var2','var3','var4','var5','var6']

In [9]:
new_var1 = []
for i in df.loc[:,'var1']:
    try:
        i = float(i)
        new_var1.append(i)
    except:
        print(i)
        new_var1.append(i)

354.1386+'[Sorting_0529_Fe Complexes .xlsx]#4 Ala2+Fe+Glu2'!$B$48:$G$692+'[Sorting_0529_Fe Complexes .xlsx]#4 Ala2+Fe+Glu2'!$B$126:$G$692
m/z


In [10]:
df['var1'] = new_var1

In [11]:
df = df.iloc[:10,:] # 10개만 test로 해보기

In [12]:
df

,var1,var2,var3,var4,var5,var6
0,85.1014,16848,16.6,643,0.3,0.0051
1,86.0599,15149,19.4,753,0.4,0.0057
2,87.0438,18891,17,663,0.3,0.0046
3,88.0755,15455,55.3,2156,1.1,0.0057
4,89.0593,17062,5.5,215,0.1,0.0052
5,89.9639,16388,25,979,0.5,0.0055
6,90.055,16786,3077.1,120565,58.7,0.0054
7,90.0913,16209,6.6,257,0.1,0.0056
8,91.058,15047,114.3,4490,2.2,0.0061
9,92.0606,16235,14.8,584,0.3,0.0057


#### 2. API를 활용해 var1의 물성에서 가능한 분자식를 찾아보자.. 세부설정은 화학지식이 필요

In [13]:
import urllib
import json
import requests

In [14]:
def findMfByMass(mass):
    chemcalcURL = 'https://www.chemcalc.org/chemcalc/em'
    mfRange = 'C0-100H0-100N0-10O0-10'
    options = {'mfRange': mfRange,
               'monoisotopicMass': mass,
              }

    data =  requests.get(chemcalcURL, options).json()
    return data

In [15]:
data = findMfByMass(df['var1'][0])

In [16]:
data = pd.DataFrame(data['results'])

In [17]:
data = data.sort_values(by = 'error',ascending= True) # 에러가 가장 작은 분자구조를 선택해보쟈
data = data.reset_index(drop = True)
mf = data.loc[0,['mf']]
mf = mf.values[0]
mf

'C3H3NO2'

In [18]:
df['var1'].values

array([85.1014, 86.0599, 87.0438, 88.0755, 89.0593, 89.9639, 90.055,
       90.0913, 91.058, 92.0606], dtype=object)

In [19]:
mf_list = []
for var1 in df['var1'].values:
    data = findMfByMass(var1)
    data = pd.DataFrame(data['results'])
    data = data.sort_values(by = 'error',ascending= True) # 에러가 가장 작은 분자구조를 선택해보쟈
    data = data.reset_index(drop = True)
    mf = data.loc[0,['mf']]
    mf = mf.values[0]
    mf_list.append(mf)

In [20]:
mf_list

['C3H3NO2',
 'C3H2O3',
 'C2HNO3',
 'C2O4',
 'C2H3NO3',
 'C2H2O4',
 'C2H2O4',
 'C2H2O4',
 'CHNO4',
 'CO5']

In [21]:
df['mf'] = mf_list

In [22]:
df

,var1,var2,var3,var4,var5,var6,mf
0,85.1014,16848,16.6,643,0.3,0.0051,C3H3NO2
1,86.0599,15149,19.4,753,0.4,0.0057,C3H2O3
2,87.0438,18891,17,663,0.3,0.0046,C2HNO3
3,88.0755,15455,55.3,2156,1.1,0.0057,C2O4
4,89.0593,17062,5.5,215,0.1,0.0052,C2H3NO3
5,89.9639,16388,25,979,0.5,0.0055,C2H2O4
6,90.055,16786,3077.1,120565,58.7,0.0054,C2H2O4
7,90.0913,16209,6.6,257,0.1,0.0056,C2H2O4
8,91.058,15047,114.3,4490,2.2,0.0061,CHNO4
9,92.0606,16235,14.8,584,0.3,0.0057,CO5


#### 3. 분자식에서 나타날 수 있는 분자구조를 찾아보쟈

In [23]:
!pip install pubchempy

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pubchempy: filename=PubChemPy-1.0.4-py3-none-any.whl size=13819 sha256=03e9953f0d755881e371ac0a686c2ae59e5992a488191ab54d645e2de3440ed9
  Stored in directory: c:\users\dongkeon\appdata\local\pip\cache\wheels\84\45\0e\b597debba098119b642eaf728ae1883d23ad8ea2a9366f2ded
Successfully built pubchempy


In [24]:
import pubchempy as pcp

In [25]:
mf = df['mf'][0]

In [26]:
mf

'C3H3NO2'

In [27]:
results = pcp.get_compounds(mf, 'formula',as_dataframe = True) # 시간 상단 소요

In [28]:
len(results.isomeric_smiles)

97

In [29]:
smiles = results.isomeric_smiles.values[0]

In [30]:
smiles

'C(C#N)C(=O)O'

In [ ]:
smiles_list = []
for mf in df['mf'].values:
    results = pcp.get_compounds(mf, 'formula',as_dataframe = True)
    smiles = results.isomeric_smiles.values[0]
    smiles_list.append(smiles)

In [ ]:
df['smiles'] = smiles_list

In [ ]:
df

#### 4. 분자구조를 그림으로 그리고 imags 폴더에 저장해보자

In [ ]:
from rdkit import Chem
from rdkit.Chem import Draw

In [ ]:
smiles = df['smiles'][0]

In [ ]:
mf = df['mf'][0]

In [ ]:
smiles

In [ ]:
mf

In [ ]:
mol = Chem.MolFromSmiles(smiles)

In [ ]:
img = Draw.MolToImage(mol,legend = mf)

In [ ]:
img

In [ ]:
img.save('data/images/{}.png'.format(mf))

In [ ]:
for idx in range(len(df)):
    smiles = df['smiles'][idx]
    mf = df['mf'][idx]
    mol = Chem.MolFromSmiles(smiles)
    img = Draw.MolToImage(mol,legend = mf)
    img.save('data/images/{}.png'.format(mf))